<a href="https://colab.research.google.com/github/hrbolek/learning/blob/master/itsystems/couchdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CouchDb Introduction

In [ ]:
import urllib.request, json 
from urllib.parse import urlencode
from base64 import b64encode
import getpass
from platform import python_version

print(python_version())
serverAddress = 'http://192.168.1.5:5984/'
userName = input('Uživatel\t:')
password = getpass.getpass('Heslo\t:')

Uživatel	:admin
Heslo	:········


## Základní metody pro práci s databází
```python
couchDbLogin   #přihlášení k databázi
```
```python
couchDbAPICall #volání api
```
Pro autentizaci čtěte:
https://docs.couchdb.org/en/stable/api/server/authn.html#api-auth-session

In [ ]:
def printJson(jsondata):
    result = json.dumps(jsondata, indent=4)
    print(result)
    #return result

cookie = ''    
def couchDbLogin(user = userName, password = password):
    global cookie

    url = serverAddress + '_session'
    data = {'username' : user, 'password': password}

    data = urllib.parse.urlencode(data).encode()
    request = urllib.request.Request(url, data = data, method = 'POST')
    response = urllib.request.urlopen(request)
    cookie = response.headers['Set-Cookie']
    cookie = cookie.split(';')[0]

    result = response.read()
    #result = response.read().decode('utf8')
    return result
    
def couchDbAPICall(location, method = 'GET', data = None):
    global cookie
    url = serverAddress + location
    encdata = None
    if not(data == None):
        encdata = json.dumps(data).encode('utf8')
        print(encdata)
    request = urllib.request.Request(url, data = encdata, method = method)
    request.add_header('Cookie', cookie)   
    result = '{}'
    print(request)

    if not(data == None):
        request.add_header('Content-Type', 'application/json')   
    try:
        response = urllib.request.urlopen(request)
        result = response.read()
    except urllib.error.HTTPError as err:
        print(err)
        print(err.reason)
        print(err.headers)
        result = err.read()
        #response = err.response
    
    #result = response.read().decode('utf8')
    return json.loads(result)

couchDbLogin()
printJson(couchDbAPICall(location = ''))

{
    "couchdb": "Welcome",
    "version": "2.3.1",
    "git_sha": "c298091a4",
    "uuid": "d2b27244ee0c9a7d5b10785985989c9a",
    "features": [
        "pluggable-storage-engines",
        "scheduler"
    ],
    "vendor": {
        "name": "The Apache Software Foundation"
    }
}


## Získání prvotních informací z databáze (metadata)

In [ ]:
data = couchDbAPICall('_session', 'GET')
print('_session')
printJson(data)
dbs = couchDbAPICall('_all_dbs', 'GET')
print('_all_dbs')
printJson(dbs)


_session
{
    "ok": true,
    "userCtx": {
        "name": null,
        "roles": []
    },
    "info": {
        "authentication_db": "_users",
        "authentication_handlers": [
            "cookie",
            "default"
        ]
    }
}
_all_dbs
[
    "_global_changes",
    "_replicator",
    "_users"
]


In [ ]:
## Vytvoření databáze

In [ ]:
dbs = couchDbAPICall('my_db', 'PUT')   #curl -X PUT http://servername:5984/my_db
print('Created database')
printJson(dbs)
dbs = couchDbAPICall('_all_dbs', 'GET')#curl -X GET http://servername:5984/_all_dbs
print('_all_dbs')
printJson(dbs)
dbs = couchDbAPICall('my_db', 'GET')   #curl -X GET http://servername:5984/next_db
print('Retrieve info about database')
printJson(dbs)
dbs = couchDbAPICall('my_db', 'DELETE')#curl -X DELETE http://servername:5984/my_db
print('Delete database')
printJson(dbs)
dbs = couchDbAPICall('_all_dbs', 'GET')#curl -X GET http://servername:5984/_all_dbs
print('_all_dbs')
printJson(dbs)


Created database
{
    "ok": true
}
_all_dbs
[
    "_global_changes",
    "_replicator",
    "_users",
    "my_db"
]
Retrieve info about database
{
    "db_name": "my_db",
    "purge_seq": "0-g1AAAAEzeJzLYWBg4MhgTmHgzcvPy09JdcjLz8gvLskBCjPlsQBJhgNA6v____ezEhnwqmuAqJuPW12SA5BMigcqwqMmAaSmHq8aiH0LIPbtJ6TuAUQdPjsVQHba41WTyJAkD1GQBQC-sGL2",
    "update_seq": "0-g1AAAAEzeJzLYWBg4MhgTmHgzcvPy09JdcjLz8gvLskBCjMlMiTJ____PyuRAYeCJAUgmWSPX40DSE08fjUJIDX1eNXksQBJhgYgBVQ2n5C6BRB1-wmpOwBRd5-QugcQdSD3ZQEAiJJi9g",
    "sizes": {
        "file": 33960,
        "external": 0,
        "active": 0
    },
    "other": {
        "data_size": 0
    },
    "doc_del_count": 0,
    "doc_count": 0,
    "disk_size": 33960,
    "disk_format_version": 7,
    "data_size": 0,
    "compact_running": false,
    "cluster": {
        "q": 8,
        "n": 1,
        "w": 1,
        "r": 1
    },
    "instance_start_time": "0"
}
Delete database
{
    "ok": true
}
_all_dbs
[
    "_global_changes",
    "_replicator",
    "_

## Nový dokument v databázi

### Vytvoření databáze pro experimenty

In [ ]:
dbs = couchDbAPICall('data', 'PUT')
printJson(dbs)

{
    "ok": true
}


### Vložení dokumentu

In [ ]:
dokument = {'data': 'informace', 'cislo': 35}
docresult = couchDbAPICall('data', 'POST', dokument)
print('result')
printJson(docresult)

b'{"data": "informace", "cislo": 35}'
result
{
    "ok": true,
    "id": "c624cce012e5f4e43856479ad9005249",
    "rev": "1-6a6fa57308075e2a9329f9aa09a19be6"
}


### Zpřístupnění vloženého dokumentu

In [ ]:
docid = docresult['id']
dbs = couchDbAPICall('data/' + docid, 'GET')
print('result')
printJson(dbs)

result
{
    "_id": "c624cce012e5f4e43856479ad9005249",
    "_rev": "1-6a6fa57308075e2a9329f9aa09a19be6",
    "data": "informace",
    "cislo": 35
}


### Seznam všech dokumentů v tabulce

In [ ]:
view = couchDbAPICall('data/_all_docs', 'GET')
print('result')
printJson(view)

result
{
    "total_rows": 3,
    "offset": 0,
    "rows": [
        {
            "id": "c624cce012e5f4e43856479ad9002ce9",
            "key": "c624cce012e5f4e43856479ad9002ce9",
            "value": {
                "rev": "1-6a6fa57308075e2a9329f9aa09a19be6"
            }
        },
        {
            "id": "c624cce012e5f4e43856479ad9003c92",
            "key": "c624cce012e5f4e43856479ad9003c92",
            "value": {
                "rev": "1-6a6fa57308075e2a9329f9aa09a19be6"
            }
        },
        {
            "id": "c624cce012e5f4e43856479ad9005249",
            "key": "c624cce012e5f4e43856479ad9005249",
            "value": {
                "rev": "1-6a6fa57308075e2a9329f9aa09a19be6"
            }
        }
    ]
}


In [ ]:
query = {'map' : 'function(doc){emit(doc.cislo)}'}
view = couchDbAPICall('data/_temp_view', 'POST', query)
print('result')
printJson(view)

b'{"map": "function(doc){emit(doc.cislo)}"}'
HTTP Error 410: Gone
Gone
Cache-Control: must-revalidate
Connection: close
Content-Length: 73
Content-Type: application/json
Date: Thu, 26 Dec 2019 20:55:38 GMT
Server: CouchDB/2.3.1 (Erlang OTP/19)
Set-Cookie: AuthSession=YWRtaW46NUUwNTFFNEE6XmT4IgtMhEDthD5vAevU-ijdwTk; Version=1; Expires=Thu, 26-Dec-2019 21:05:38 GMT; Max-Age=600; Path=/; HttpOnly
X-Couch-Request-ID: 5243361bd5
X-CouchDB-Body-Time: 0


result
{
    "error": "gone",
    "reason": "Temporary views are not supported in CouchDB"
}


### Odstranění databáze

In [ ]:
dbs = couchDbAPICall('data', 'DELETE')
printJson(dbs)

{
    "ok": true
}
